In [29]:
import os
from sys import maxsize
import numpy as np
import matplotlib.pyplot as plt

100 per file * 10 files

In [31]:
def readdat(directory):
    Us = []
    Vs = []
    As = []
    
    for filenum in range(10):
        print(directory + filename_pre + str(filenum))

        with open(directory + filename_pre + str(filenum), 'r') as file:
            for c in range(100):
                line = file.readline().split()
                # print(line)
                u_size = (int(line[0]), int(line[1]))
                utemp = []
                for i in range(u_size[0]):
                    utemp.append([float(x) for x in file.readline().split()])
                Us.append(utemp)
                # print(utemp)

                line = file.readline().split()
                # print(line)
                v_size = (int(line[0]), int(line[1]))
                vtemp = []
                for i in range(v_size[0]):
                    vtemp.append([float(x) for x in file.readline().split()])
                Vs.append(vtemp)
                # print(vtemp)

                line = file.readline().split()
                # print(line)
                alloc_size = (int(line[0]), int(line[1]))
                alloctemp = []
                for i in range(alloc_size[0]):
                    alloctemp.append([int(x) for x in file.readline().split()])
                As.append(alloctemp)
                # print(alloctemp)

                line = file.readline()
                # print(line)
    return (Us, Vs, As)

EF calculations

In [32]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        pairwise_alloc.append(alloc[i] & alloc[i+1])
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

def envyMap(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
    return envy_map

def envyMap_upto1(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

def envyMap_upto2(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                        if envy_map[i][j] < 0 :
                            envy_res_drop2_j = np.zeros((size_res, size_res))
                            max_pos_envy_drop2 = - maxsize
                            for p in range(len(envy_res_drop2_j)):
                                for q in range(len(envy_res_drop2_j[p])):
                                    if p == q:
                                        continue
                                    new_alloc_j_drop2 = list(allocs[j])
                                    new_alloc_j_drop2[p] = 0
                                    new_alloc_j_drop2[q] = 0
                                    envy_res_drop2_j[p][q] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                                    - complementing_utility(new_alloc_j_drop2, utilities[i], p_utilities[i])
                                    if envy_res_drop2_j[p][q] >= max_pos_envy_drop2:
                                        envy_map[i][j] = envy_res_drop2_j[p][q]
                                        max_pos_envy_drop2 = envy_res_drop2_j[p][q]
                
    return envy_map

In [54]:
def ef_percent(dir_name):
    Us, Vs, As = readdat(dir_name)
    Us = np.array(Us)
    Vs = np.array(Vs)
    As = np.array(As)
    print("Us shape :", Us.shape)
    print("Vs shape :", Vs.shape)
    print("As shape :", As.shape)
    
    ef_map = []
    envymap0 = []
    for i in range(1000):
        envymaptemp = envyMap(As[i], Us[i], Vs[i])
        envymap0.append(envymaptemp)
        # print(envymaptemp)
        ef_map.append(not bool(np.sum(envymaptemp<0)))

    ef_map = np.array(ef_map)
    # print(ef_map)
    print("EF0 percent : ", np.sum(ef_map)/len(ef_map) * 100)

    ef1_map = []
    envymap1 = []
    for i in range(1000):
        envymaptemp = envyMap_upto1(As[i], Us[i], Vs[i])
        envymap1.append(envymaptemp)
        # print(envymaptemp)
        ef1_map.append(not bool(np.sum(envymaptemp<0)))

    ef1_map = np.array(ef1_map)
    # print(ef_map)
    print("EF1 percent : ", np.sum(ef1_map)/len(ef1_map) * 100)

    ef2_map = []
    envymap2 = []
    for i in range(1000):
        envymaptemp = envyMap_upto2(As[i], Us[i], Vs[i])
        envymap2.append(envymaptemp)
        # print(envymaptemp)
        ef2_map.append(not bool(np.sum(envymaptemp<0)))

    ef2_map = np.array(ef2_map)
    # print(ef_map)
    print("EF2 percent : ", np.sum(ef2_map)/len(ef2_map) * 100)


For additive utilities

In [49]:
dir_u_r10_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/"
dir_uv_r10_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/"

dir_u_r4_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/"
dir_uv_r4_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/"

dir_u_r6_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/"
dir_uv_r6_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/"

dir_u_r8_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r8_u2/" # Not yet computed
dir_uv_r8_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/"

filename_pre = "nash_allocs_random_uv_1"

For complementing utilities

In [45]:
# failtemp = np.argwhere(ef1_map==False)[0][0]
# print(failtemp)
# print(Us[failtemp])
# print(Vs[failtemp])
# print(As[failtemp])
# print(envymap1[failtemp])

In [55]:
ef_percent(dir_u_r10_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
A

In [63]:
# ef_percent(dir_u_r8_u2)

In [58]:
ef_percent(dir_u_r6_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 6)
Vs shape : (1000, 2, 3)
As shape : (

In [65]:
ef_percent(dir_u_r4_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 4)
Vs shape : (1000, 2, 2)
As shape : (

In [69]:
ef_percent(dir_uv_r10_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 10)
Vs shape : (100

In [70]:
ef_percent(dir_uv_r8_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 8)
Vs shape : (1000, 2, 4)
As

In [71]:
ef_percent(dir_uv_r6_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 6)
Vs shape : (1000, 2, 3)
As

In [72]:
ef_percent(dir_uv_r4_u2)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/nash_allocs_random_uv_19
Us shape : (1000, 2, 4)
Vs shape : (1000, 2, 2)
As